In [1]:
import argparse
import os
import torch
from tqdm import tqdm
import pdb
import numpy as np

import matplotlib.pyplot as plt
from utils import setup_seed, keep_bbox_from_image_range, \
    keep_bbox_from_lidar_range, write_pickle, write_label, \
    iou2d, iou3d_camera, iou_bev
from dataset import Kitti, get_dataloader
from model import PointPillars
from loss import Loss
from torch.utils.tensorboard import SummaryWriter

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def get_score_thresholds(tp_scores, total_num_valid_gt, num_sample_pts=41):
    score_thresholds = []
    tp_scores = sorted(tp_scores)[::-1]
    cur_recall, pts_ind = 0, 0
    for i, score in enumerate(tp_scores):
        lrecall = (i + 1) / total_num_valid_gt
        rrecall = (i + 2) / total_num_valid_gt

        if i == len(tp_scores) - 1:
            score_thresholds.append(score)
            break

        if (lrecall + rrecall) / 2 < cur_recall:
            continue

        score_thresholds.append(score)
        pts_ind += 1
        cur_recall = pts_ind / (num_sample_pts - 1)
    return score_thresholds

In [3]:
def do_eval(det_results, gt_results, CLASSES, saved_path):
    '''
    det_results: list,
    gt_results: dict(id -> det_results)
    CLASSES: dict
    '''
    assert len(det_results) == len(gt_results)
    f = open(os.path.join(saved_path, 'eval_results.txt'), 'w')

    # 1. calculate iou
    ious = {
        'bbox_2d': [],
        'bbox_bev': [],
        'bbox_3d': []
    }
    ids = list(sorted(gt_results.keys()))
    for id in ids:
        gt_result = gt_results[id]['annos']
        det_result = det_results[id]

        # 1.1, 2d bboxes iou
        gt_bboxes2d = gt_result['bbox'].astype(np.float32)
        det_bboxes2d = det_result['bbox'].astype(np.float32)
        iou2d_v = iou2d(torch.from_numpy(gt_bboxes2d).cuda(), torch.from_numpy(det_bboxes2d).cuda())
        ious['bbox_2d'].append(iou2d_v.cpu().numpy())

        # 1.2, bev iou
        gt_location = gt_result['location'].astype(np.float32)
        gt_dimensions = gt_result['dimensions'].astype(np.float32)
        gt_rotation_y = gt_result['rotation_y'].astype(np.float32)
        det_location = det_result['location'].astype(np.float32)
        det_dimensions = det_result['dimensions'].astype(np.float32)
        det_rotation_y = det_result['rotation_y'].astype(np.float32)

        gt_bev = np.concatenate([gt_location[:, [0, 2]], gt_dimensions[:, [0, 2]], gt_rotation_y[:, None]], axis=-1)
        det_bev = np.concatenate([det_location[:, [0, 2]], det_dimensions[:, [0, 2]], det_rotation_y[:, None]], axis=-1)
        iou_bev_v = iou_bev(torch.from_numpy(gt_bev).cuda(), torch.from_numpy(det_bev).cuda())
        ious['bbox_bev'].append(iou_bev_v.cpu().numpy())

        # 1.3, 3dbboxes iou
        gt_bboxes3d = np.concatenate([gt_location, gt_dimensions, gt_rotation_y[:, None]], axis=-1)
        det_bboxes3d = np.concatenate([det_location, det_dimensions, det_rotation_y[:, None]], axis=-1)
        iou3d_v = iou3d_camera(torch.from_numpy(gt_bboxes3d).cuda(), torch.from_numpy(det_bboxes3d).cuda())
        ious['bbox_3d'].append(iou3d_v.cpu().numpy())

    MIN_IOUS = {
        'Pedestrian': [0.5, 0.5, 0.5],
        'Cyclist': [0.5, 0.5, 0.5],
        'Car': [0.7, 0.7, 0.7]
    }
    MIN_HEIGHT = [40, 25, 25]

    overall_results = {}
    for e_ind, eval_type in enumerate(['bbox_2d', 'bbox_bev', 'bbox_3d']):
        eval_ious = ious[eval_type]
        eval_ap_results, eval_aos_results = {}, {}
        for cls in CLASSES:
            eval_ap_results[cls] = []
            eval_aos_results[cls] = []
            CLS_MIN_IOU = MIN_IOUS[cls][e_ind]
            for difficulty in [0, 1, 2]:
                # 1. bbox property
                total_gt_ignores, total_det_ignores, total_dc_bboxes, total_scores = [], [], [], []
                total_gt_alpha, total_det_alpha = [], []
                for id in ids:
                    gt_result = gt_results[id]['annos']
                    det_result = det_results[id]

                    # 1.1 gt bbox property
                    cur_gt_names = gt_result['name']
                    cur_difficulty = gt_result['difficulty']
                    gt_ignores, dc_bboxes = [], []
                    for j, cur_gt_name in enumerate(cur_gt_names):
                        ignore = cur_difficulty[j] < 0 or cur_difficulty[j] > difficulty
                        if cur_gt_name == cls:
                            valid_class = 1
                        elif cls == 'Pedestrian' and cur_gt_name == 'Person_sitting':
                            valid_class = 0
                        elif cls == 'Car' and cur_gt_name == 'Van':
                            valid_class = 0
                        else:
                            valid_class = -1
                        
                        if valid_class == 1 and not ignore:
                            gt_ignores.append(0)
                        elif valid_class == 0 or (valid_class == 1 and ignore):
                            gt_ignores.append(1)
                        else:
                            gt_ignores.append(-1)
                        
                        if cur_gt_name == 'DontCare':
                            dc_bboxes.append(gt_result['bbox'][j])
                    total_gt_ignores.append(gt_ignores)
                    total_dc_bboxes.append(np.array(dc_bboxes))
                    total_gt_alpha.append(gt_result['alpha'])

                    # 1.2 det bbox property
                    cur_det_names = det_result['name']
                    cur_det_heights = det_result['bbox'][:, 3] - det_result['bbox'][:, 1]
                    det_ignores = []
                    for j, cur_det_name in enumerate(cur_det_names):
                        if cur_det_heights[j] < MIN_HEIGHT[difficulty]:
                            det_ignores.append(1)
                        elif cur_det_name == cls:
                            det_ignores.append(0)
                        else:
                            det_ignores.append(-1)
                    total_det_ignores.append(det_ignores)
                    total_scores.append(det_result['score'])
                    total_det_alpha.append(det_result['alpha'])

                # 2. calculate scores thresholds for PR curve
                tp_scores = []
                for i, id in enumerate(ids):
                    cur_eval_ious = eval_ious[i]
                    gt_ignores, det_ignores = total_gt_ignores[i], total_det_ignores[i]
                    scores = total_scores[i]

                    nn, mm = cur_eval_ious.shape
                    assigned = np.zeros((mm, ), dtype=np.bool_)
                    for j in range(nn):
                        if gt_ignores[j] == -1:
                            continue
                        match_id, match_score = -1, -1
                        for k in range(mm):
                            if not assigned[k] and det_ignores[k] >= 0 and cur_eval_ious[j, k] > CLS_MIN_IOU and scores[k] > match_score:
                                match_id = k
                                match_score = scores[k]
                        if match_id != -1:
                            assigned[match_id] = True
                            if det_ignores[match_id] == 0 and gt_ignores[j] == 0:
                                tp_scores.append(match_score)
                total_num_valid_gt = np.sum([np.sum(np.array(gt_ignores) == 0) for gt_ignores in total_gt_ignores])
                score_thresholds = get_score_thresholds(tp_scores, total_num_valid_gt)    
            
                # 3. draw PR curve and calculate mAP
                tps, fns, fps, total_aos = [], [], [], []

                for score_threshold in score_thresholds:
                    tp, fn, fp = 0, 0, 0
                    aos = 0
                    for i, id in enumerate(ids):
                        cur_eval_ious = eval_ious[i]
                        gt_ignores, det_ignores = total_gt_ignores[i], total_det_ignores[i]
                        gt_alpha, det_alpha = total_gt_alpha[i], total_det_alpha[i]
                        scores = total_scores[i]

                        nn, mm = cur_eval_ious.shape
                        assigned = np.zeros((mm, ), dtype=np.bool_)
                        for j in range(nn):
                            if gt_ignores[j] == -1:
                                continue
                            match_id, match_iou = -1, -1
                            for k in range(mm):
                                if not assigned[k] and det_ignores[k] >= 0 and scores[k] >= score_threshold and cur_eval_ious[j, k] > CLS_MIN_IOU:
    
                                    if det_ignores[k] == 0 and cur_eval_ious[j, k] > match_iou:
                                        match_iou = cur_eval_ious[j, k]
                                        match_id = k
                                    elif det_ignores[k] == 1 and match_iou == -1:
                                        match_id = k

                            if match_id != -1:
                                assigned[match_id] = True
                                if det_ignores[match_id] == 0 and gt_ignores[j] == 0:
                                    tp += 1
                                    if eval_type == 'bbox_2d':
                                        aos += (1 + np.cos(gt_alpha[j] - det_alpha[match_id])) / 2
                            else:
                                if gt_ignores[j] == 0:
                                    fn += 1
                            
                        for k in range(mm):
                            if det_ignores[k] == 0 and scores[k] >= score_threshold and not assigned[k]:
                                fp += 1
                        
                        # In case 2d bbox evaluation, we should consider dontcare bboxes
                        if eval_type == 'bbox_2d':
                            dc_bboxes = total_dc_bboxes[i]
                            det_bboxes = det_results[id]['bbox']
                            if len(dc_bboxes) > 0:
                                ious_dc_det = iou2d(torch.from_numpy(det_bboxes), torch.from_numpy(dc_bboxes), metric=1).numpy().T
                                for j in range(len(dc_bboxes)):
                                    for k in range(len(det_bboxes)):
                                        if det_ignores[k] == 0 and scores[k] >= score_threshold and not assigned[k]:
                                            if ious_dc_det[j, k] > CLS_MIN_IOU:
                                                fp -= 1
                                                assigned[k] = True
                            
                    tps.append(tp)
                    fns.append(fn)
                    fps.append(fp)
                    if eval_type == 'bbox_2d':
                        total_aos.append(aos)

                tps, fns, fps = np.array(tps), np.array(fns), np.array(fps)

                precisions = tps / (tps + fns) # actually this is recalls
                # precisions = tps / (tps + fps)
                for i in range(len(score_thresholds)):
                    precisions[i] = np.max(precisions[i:])
                
                sums_AP = 0
                for i in range(0, len(score_thresholds), 4):
                    sums_AP += precisions[i]
                mAP = sums_AP / 11 * 100
                eval_ap_results[cls].append(mAP)

                if eval_type == 'bbox_2d':
                    total_aos = np.array(total_aos)
                    similarity = total_aos / (tps + fps)
                    for i in range(len(score_thresholds)):
                        similarity[i] = np.max(similarity[i:])
                    sums_similarity = 0
                    for i in range(0, len(score_thresholds), 4):
                        sums_similarity += similarity[i]
                    mSimilarity = sums_similarity / 11 * 100
                    eval_aos_results[cls].append(mSimilarity)

        print(f'=========={eval_type.upper()}==========')
        print(f'=========={eval_type.upper()}==========', file=f)
        for k, v in eval_ap_results.items():
            print(f'{k} AP@{MIN_IOUS[k][e_ind]}: {v[0]:.4f} {v[1]:.4f} {v[2]:.4f}')
            print(f'{k} AP@{MIN_IOUS[k][e_ind]}: {v[0]:.4f} {v[1]:.4f} {v[2]:.4f}', file=f)
        if eval_type == 'bbox_2d':
            print(f'==========AOS==========')
            print(f'==========AOS==========', file=f)
            for k, v in eval_aos_results.items():
                print(f'{k} AOS@{MIN_IOUS[k][e_ind]}: {v[0]:.4f} {v[1]:.4f} {v[2]:.4f}')
                print(f'{k} AOS@{MIN_IOUS[k][e_ind]}: {v[0]:.4f} {v[1]:.4f} {v[2]:.4f}', file=f)
        
        overall_results[eval_type] = np.mean(list(eval_ap_results.values()), 0)
        if eval_type == 'bbox_2d':
            overall_results['AOS'] = np.mean(list(eval_aos_results.values()), 0)
    
    print(f'\n==========Overall==========')
    print(f'\n==========Overall==========', file=f)
    for k, v in overall_results.items():
        print(f'{k} AP: {v[0]:.4f} {v[1]:.4f} {v[2]:.4f}')
        print(f'{k} AP: {v[0]:.4f} {v[1]:.4f} {v[2]:.4f}', file=f)
    f.close()
    

### Evaluation Arguments

In [4]:
class Args:
    def __init__(self):
        self.data_root = "dataset/KITTI"
        self.ckpt = "logs/pillar_sequence_memory_gating_logloss/checkpoints/epoch_160.pth"
        self.saved_path = "logs/pillar_sequence_memory_gating_logloss/results"
        self.batch_size = 1
        self.num_workers = 4
        self.nclasses = 3
        self.no_cuda = not torch.cuda.is_available()

args = Args()


### Dataloader

In [5]:
val_dataset = Kitti(data_root=args.data_root,
                    split='val')
val_dataloader = get_dataloader(dataset=val_dataset, 
                                batch_size=args.batch_size, 
                                num_workers=args.num_workers,
                                shuffle=False)
CLASSES = Kitti.CLASSES
LABEL2CLASSES = {v:k for k, v in CLASSES.items()}

### Model

In [6]:
if not args.no_cuda:
    model = PointPillars(nclasses=args.nclasses).cuda()
    model.load_state_dict(torch.load(args.ckpt))
else:
    model = PointPillars(nclasses=args.nclasses)
    model.load_state_dict(
        torch.load(args.ckpt, map_location=torch.device('cpu')))
    


saved_path = args.saved_path
os.makedirs(saved_path, exist_ok=True)
saved_submit_path = os.path.join(saved_path, 'submit')
os.makedirs(saved_submit_path, exist_ok=True)

pcd_limit_range = np.array([0, -40, -3, 70.4, 40, 0.0], dtype=np.float32)

model.eval()

PointPillars(
  (pillar_layer): PillarLayer(
    (voxel_layer): Voxelization(voxel_size=[0.16, 0.16, 4], point_cloud_range=[0, -39.68, -3, 69.12, 39.68, 1], max_num_points=32, max_voxels=(16000, 40000), deterministic=True)
  )
  (pillar_encoder): PillarEncoder(
    (conv): Conv1d(9, 64, kernel_size=(1,), stride=(1,), bias=False)
    (bn): BatchNorm1d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
  )
  (gate0): Gate(
    (conv1): Conv2d(128, 1, kernel_size=(1, 1), stride=(1, 1))
    (avgpool): AdaptiveAvgPool2d(output_size=1)
    (sigmoid): Sigmoid()
  )
  (gate1): Gate(
    (conv1): Conv2d(128, 1, kernel_size=(1, 1), stride=(1, 1))
    (avgpool): AdaptiveAvgPool2d(output_size=1)
    (sigmoid): Sigmoid()
  )
  (gate2): Gate(
    (conv1): Conv2d(128, 1, kernel_size=(1, 1), stride=(1, 1))
    (avgpool): AdaptiveAvgPool2d(output_size=1)
    (sigmoid): Sigmoid()
  )
  (gate3): Gate(
    (conv1): Conv2d(128, 1, kernel_size=(1, 1), stride=(1, 1))
    (avgpool): Adaptive

### Evaluation

In [7]:
val_epoch_compute = 0
val_epoch_g_probability = 0

with torch.no_grad():
    format_results = {}
    print('Predicting and Formatting the results.')
    for i, data_dict in enumerate(tqdm(val_dataloader)):
        if not args.no_cuda:
            # move the tensors to the cuda
            for key in data_dict:
                for j, item in enumerate(data_dict[key]):
                    if torch.is_tensor(item):
                        data_dict[key][j] = data_dict[key][j].cuda()
        
        batched_pts = data_dict['batched_pts']
        batched_pts0 = data_dict['batched_pts0']
        batched_gt_bboxes = data_dict['batched_gt_bboxes']
        batched_labels = data_dict['batched_labels']
        batched_difficulty = data_dict['batched_difficulty']
        batch_results0, f = model(batched_pts=batched_pts,
                               batched_pts0=batched_pts0, 
                                mode='val',
                                batched_gt_bboxes=batched_gt_bboxes, 
                                batched_gt_labels=batched_labels)
        # pdb.set_trace()
        val_epoch_compute = val_epoch_compute + f.shape[0]*f.shape[1]
        val_epoch_g_probability = val_epoch_g_probability + torch.sum(f>0.5)

        for j, result in enumerate(batch_results0):
            format_result = {
                'name': [],
                'truncated': [],
                'occluded': [],
                'alpha': [],
                'bbox': [],
                'dimensions': [],
                'location': [],
                'rotation_y': [],
                'score': []
            }
            
            calib_info = data_dict['batched_calib_info'][j]
            tr_velo_to_cam = calib_info['Tr_velo_to_cam'].astype(np.float32)
            r0_rect = calib_info['R0_rect'].astype(np.float32)
            P2 = calib_info['P2'].astype(np.float32)
            image_shape = data_dict['batched_img_info'][j]['image_shape']
            idx = data_dict['batched_img_info'][j]['image_idx']
            result_filter = keep_bbox_from_image_range(result, tr_velo_to_cam, r0_rect, P2, image_shape)
            result_filter = keep_bbox_from_lidar_range(result_filter, pcd_limit_range)

            lidar_bboxes = result_filter['lidar_bboxes']
            labels, scores = result_filter['labels'], result_filter['scores']
            bboxes2d, camera_bboxes = result_filter['bboxes2d'], result_filter['camera_bboxes']
            for lidar_bbox, label, score, bbox2d, camera_bbox in \
                zip(lidar_bboxes, labels, scores, bboxes2d, camera_bboxes):
                format_result['name'].append(LABEL2CLASSES[label])
                format_result['truncated'].append(0.0)
                format_result['occluded'].append(0)
                alpha = camera_bbox[6] - np.arctan2(camera_bbox[0], camera_bbox[2])
                format_result['alpha'].append(alpha)
                format_result['bbox'].append(bbox2d)
                format_result['dimensions'].append(camera_bbox[3:6])
                format_result['location'].append(camera_bbox[:3])
                format_result['rotation_y'].append(camera_bbox[6])
                format_result['score'].append(score)
            
            write_label(format_result, os.path.join(saved_submit_path, f'{idx:06d}.txt'))

            format_results[idx] = {k:np.array(v) for k, v in format_result.items()}
    
    write_pickle(format_results, os.path.join(saved_path, 'results.pkl'))

print(val_epoch_compute)
print(val_epoch_g_probability)
print('Evaluating.. Please wait several seconds.')
do_eval(format_results, val_dataset.data_infos, CLASSES, saved_path)

Predicting and Formatting the results.


  0%|          | 0/2879 [00:00<?, ?it/s]

/home/sayeed/anaconda3/envs/openmmlab/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525541702/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 2879/2879 [04:35<00:00, 10.46it/s]


23032
tensor(20153, device='cuda:0')
Evaluating.. Please wait several seconds.
==========BBOX_2D==========
Pedestrian AP@0.5: 55.4703 52.8956 50.9357
Cyclist AP@0.5: 99.1886 84.0851 81.4598
Car AP@0.7: 84.5046 71.1800 67.5895
==========AOS==========
Pedestrian AOS@0.5: 47.5685 46.7146 45.4635
Cyclist AOS@0.5: 77.3289 63.2157 61.2804
Car AOS@0.7: 84.3903 77.6703 76.4459
==========BBOX_BEV==========
Pedestrian AP@0.5: 63.9996 54.4097 52.0281
Cyclist AP@0.5: 97.9716 71.6169 68.6217
Car AP@0.7: 83.2078 69.0645 65.2654
==========BBOX_3D==========
Pedestrian AP@0.5: 53.5138 51.1675 42.5835
Cyclist AP@0.5: 86.1147 65.2672 55.6099
Car AP@0.7: 57.1114 43.2868 39.8667

==========Overall==========
bbox_2d AP: 79.7212 69.3869 66.6616
AOS AP: 69.7626 62.5335 61.0633
bbox_bev AP: 81.7263 65.0304 61.9717
bbox_3d AP: 65.5800 53.2405 46.0200


In [ ]:
# with torch.no_grad():
#     format_results = {}
#     print('Predicting and Formatting the results.')
#     for i, data_dict in enumerate(tqdm(val_dataloader)):
#         if not args.no_cuda:
#             # move the tensors to the cuda
#             for key in data_dict:
#                 for j, item in enumerate(data_dict[key]):
#                     if torch.is_tensor(item):
#                         data_dict[key][j] = data_dict[key][j].cuda()
        
#         batched_pts = data_dict['batched_pts']
#         batched_gt_bboxes = data_dict['batched_gt_bboxes']
#         batched_labels = data_dict['batched_labels']
#         batched_difficulty = data_dict['batched_difficulty']
#         batch_results0, batch_results1, batch_results2 = model(batched_pts=batched_pts, 
#                                 mode='val',
#                                 batched_gt_bboxes=batched_gt_bboxes, 
#                                 batched_gt_labels=batched_labels)
#         # pdb.set_trace()

#         for j, result in enumerate(batch_results2):
#             format_result = {
#                 'name': [],
#                 'truncated': [],
#                 'occluded': [],
#                 'alpha': [],
#                 'bbox': [],
#                 'dimensions': [],
#                 'location': [],
#                 'rotation_y': [],
#                 'score': []
#             }
            
#             calib_info = data_dict['batched_calib_info'][j]
#             tr_velo_to_cam = calib_info['Tr_velo_to_cam'].astype(np.float32)
#             r0_rect = calib_info['R0_rect'].astype(np.float32)
#             P2 = calib_info['P2'].astype(np.float32)
#             image_shape = data_dict['batched_img_info'][j]['image_shape']
#             idx = data_dict['batched_img_info'][j]['image_idx']
#             result_filter = keep_bbox_from_image_range(result, tr_velo_to_cam, r0_rect, P2, image_shape)
#             result_filter = keep_bbox_from_lidar_range(result_filter, pcd_limit_range)

#             lidar_bboxes = result_filter['lidar_bboxes']
#             labels, scores = result_filter['labels'], result_filter['scores']
#             bboxes2d, camera_bboxes = result_filter['bboxes2d'], result_filter['camera_bboxes']
#             for lidar_bbox, label, score, bbox2d, camera_bbox in \
#                 zip(lidar_bboxes, labels, scores, bboxes2d, camera_bboxes):
#                 format_result['name'].append(LABEL2CLASSES[label])
#                 format_result['truncated'].append(0.0)
#                 format_result['occluded'].append(0)
#                 alpha = camera_bbox[6] - np.arctan2(camera_bbox[0], camera_bbox[2])
#                 format_result['alpha'].append(alpha)
#                 format_result['bbox'].append(bbox2d)
#                 format_result['dimensions'].append(camera_bbox[3:6])
#                 format_result['location'].append(camera_bbox[:3])
#                 format_result['rotation_y'].append(camera_bbox[6])
#                 format_result['score'].append(score)
            
#             write_label(format_result, os.path.join(saved_submit_path, f'{idx:06d}.txt'))

#             format_results[idx] = {k:np.array(v) for k, v in format_result.items()}
    
#     write_pickle(format_results, os.path.join(saved_path, 'results.pkl'))

# print('Evaluating.. Please wait several seconds.')
# do_eval(format_results, val_dataset.data_infos, CLASSES, saved_path)

# Test

In [ ]:
import argparse
import cv2
import numpy as np
import os
import torch
import pdb
from tqdm import tqdm 


from utils import setup_seed, read_points, read_calib, read_label, \
    keep_bbox_from_image_range, keep_bbox_from_lidar_range, vis_pc, \
    vis_img_3d, bbox3d2corners_camera, points_camera2image, \
    bbox_camera2lidar
from model import PointPillars



def point_range_filter(pts, point_range=[0, -39.68, -3, 69.12, 39.68, 1]):
    '''
    data_dict: dict(pts, gt_bboxes_3d, gt_labels, gt_names, difficulty)
    point_range: [x1, y1, z1, x2, y2, z2]
    '''
    flag_x_low = pts[:, 0] > point_range[0]
    flag_y_low = pts[:, 1] > point_range[1]
    flag_z_low = pts[:, 2] > point_range[2]
    flag_x_high = pts[:, 0] < point_range[3]
    flag_y_high = pts[:, 1] < point_range[4]
    flag_z_high = pts[:, 2] < point_range[5]
    keep_mask = flag_x_low & flag_y_low & flag_z_low & flag_x_high & flag_y_high & flag_z_high
    pts = pts[keep_mask]
    return pts 

In [ ]:
class TestArgs:
    def __init__(self, fileName):
        self.data_root = "dataset/KITTI"
        self.ckpt = "logs/pillar_sequence_memory/checkpoints/epoch_160.pth"
        self.pc_path = self.data_root + "/training/velodyne/" + fileName + ".bin"
        self.calib_path = self.data_root + "/training/calib/" + fileName + ".txt"
        self.img_path = self.data_root + "/training/image_2/" + fileName + ".png"
        self.gt_path = self.data_root + "/training/label_2/" + fileName + ".txt"
        self.no_cuda = not torch.cuda.is_available()

args = TestArgs("0000_000020")


In [ ]:
CLASSES = {
    'Pedestrian': 0, 
    'Cyclist': 1, 
    'Car': 2
    }
LABEL2CLASSES = {v:k for k, v in CLASSES.items()}
pcd_limit_range = np.array([0, -40, -3, 70.4, 40, 0.0], dtype=np.float32)

if not args.no_cuda:
    model = PointPillars(nclasses=len(CLASSES)).cuda()
    model.load_state_dict(torch.load(args.ckpt))
else:
    model = PointPillars(nclasses=len(CLASSES))
    model.load_state_dict(
        torch.load(args.ckpt, map_location=torch.device('cpu')))
    
model.eval()

In [ ]:

# Open the file

output_destination = "outputs/matryoshka_3/"

if not os.path.exists(output_destination):
    os.makedirs(output_destination)
    
with open('dataset/ImageSets/val.txt', 'r') as file:
    # Read the lines into a list
    filenames = file.read().splitlines()

for filename in tqdm(filenames):

    args = TestArgs(filename)

    pc = read_points(args.pc_path)
    pc = point_range_filter(pc)
    pc_torch = torch.from_numpy(pc)

        
    with torch.no_grad():
        if not args.no_cuda:
            pc_torch = pc_torch.cuda()
        
        result_filter0, result_filter1, result_filter2 = model(batched_pts=[pc_torch], 
                                mode='test')
        results = [result_filter0, result_filter1, result_filter2]

    for model_no in range(3):
        result_filter = results[model_no][0]

        img = cv2.imread(args.img_path, 1)
        gt_label = read_label(args.gt_path)
        calib_info = read_calib(args.calib_path)

        tr_velo_to_cam = calib_info['Tr_velo_to_cam'].astype(np.float32)
        r0_rect = calib_info['R0_rect'].astype(np.float32)
        P2 = calib_info['P2'].astype(np.float32)

        image_shape = img.shape[:2]
        result_filter = keep_bbox_from_image_range(result_filter, tr_velo_to_cam, r0_rect, P2, image_shape)

        result_filter = keep_bbox_from_lidar_range(result_filter, pcd_limit_range)
        lidar_bboxes = result_filter['lidar_bboxes']
        labels, scores = result_filter['labels'], result_filter['scores']

        # vis_pc(pc, bboxes=lidar_bboxes, labels=labels)

        bboxes2d, camera_bboxes = result_filter['bboxes2d'], result_filter['camera_bboxes'] 
        bboxes_corners = bbox3d2corners_camera(camera_bboxes)
        image_points = points_camera2image(bboxes_corners, P2)
        img = vis_img_3d(img, image_points, labels, rt=True)

        tr_velo_to_cam = calib_info['Tr_velo_to_cam'].astype(np.float32)
        r0_rect = calib_info['R0_rect'].astype(np.float32)

        dimensions = gt_label['dimensions']
        location = gt_label['location']
        rotation_y = gt_label['rotation_y']
        gt_labels = np.array([CLASSES.get(item, -1) for item in gt_label['name']])
        sel = gt_labels != -1
        gt_labels = gt_labels[sel]
        bboxes_camera = np.concatenate([location, dimensions, rotation_y[:, None]], axis=-1)
        gt_lidar_bboxes = bbox_camera2lidar(bboxes_camera, tr_velo_to_cam, r0_rect)
        bboxes_camera = bboxes_camera[sel]
        gt_lidar_bboxes = gt_lidar_bboxes[sel]

        gt_labels = [-1] * len(gt_label['name']) # to distinguish between the ground truth and the predictions
        
        pred_gt_lidar_bboxes = np.concatenate([lidar_bboxes, gt_lidar_bboxes], axis=0)
        pred_gt_labels = np.concatenate([labels, gt_labels])
        # vis_pc(pc, pred_gt_lidar_bboxes, labels=pred_gt_labels)

        bboxes_corners = bbox3d2corners_camera(bboxes_camera)
        image_points = points_camera2image(bboxes_corners, P2)
        gt_labels = [-1] * len(gt_label['name'])
        img = vis_img_3d(img, image_points, gt_labels, rt=True)

        cv2.imwrite(output_destination + "det" + str(model_no)+ "_" + filename + ".png", img)
        # break

In [ ]:
output_destination = "outputs/matryoshka_merge/"

if not os.path.exists(output_destination):
    os.makedirs(output_destination)
    
with open('dataset/ImageSets/val.txt', 'r') as file:
    # Read the lines into a list
    filenames = file.read().splitlines()

for filename in tqdm(filenames):
    im0 = cv2.imread("outputs/matryoshka_3/det0_"+ filename+ ".png")
    im1 = cv2.imread("outputs/matryoshka_3/det1_"+ filename+ ".png")
    im2 = cv2.imread("outputs/matryoshka_3/det2_"+ filename+ ".png")

    im = cv2.vconcat([im0, im1, im2])
    cv2.imwrite(output_destination + filename + ".png", im)

## Energy Test

In [ ]:
# Open the file

output_destination = "outputs/matryoshka_energy/"

if not os.path.exists(output_destination):
    os.makedirs(output_destination)
    
with open('dataset/ImageSets/val.txt', 'r') as file:
    # Read the lines into a list
    filenames = file.read().splitlines()

i = 0
for filename in tqdm(filenames):

    i = i+1
    args = TestArgs(filename)

    pc = read_points(args.pc_path)
    pc = point_range_filter(pc)
    pc_torch = torch.from_numpy(pc)

        
    with torch.no_grad():
        if not args.no_cuda:
            pc_torch = pc_torch.cuda()
        
        result_filter0, result_filter1, result_filter2, features = model(batched_pts=[pc_torch], 
                                mode='test')
        # results = [result_filter0, result_filter1, result_filter2]
        features_np = np.squeeze(features.detach().cpu().numpy())

        features_power = np.linalg.norm(features_np, ord=2, axis=(1,2))

        plt.plot(features_power)

    
        # cv2.imwrite(output_destination + "det" + str(model_no)+ "_" + filename + ".png", img)

    if i==20:
        break

In [ ]:
import matplotlib.pyplot as plt

features_np = np.squeeze(features.detach().cpu().numpy())

features_power = np.linalg.norm(features_np, ord=2, axis=(1,2))

plt.plot(features_power)
# for i in range(64):
#     plt.imshow(features_np[i, :, :])
#     plt.show()